In [ ]:
import cv2
import numpy as np
import time #to sh0w frame rate
import os
# %run ./HandTrackingModule.ipynb
import HandTrackingModule as htm
brushThickness=15
eraserThickness=100

folderPath="ChooseBar"
myList=os.listdir(folderPath)
print(myList)
#importing images
overlayList=[]
for imPath in myList:
    image=cv2.imread(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))
header=overlayList[0]
drawColor=(23,23,255)
cap=cv2.VideoCapture(0)
cap.set(3,1280)   #3=propertyid->width
cap.set(4,720)    #4=propertyid->height
detector=htm.handDetector(min_detection_confidence=0.85)
xp,yp=0,0
imgCanvas=np.zeros((720,1280,3),np.uint8)
while True:
    
    #1.Import image
    success,img=cap.read()
    img=cv2.flip(img,1)
    #2.Finding hand landmarks
    img=detector.findHands(img)
    lmList=detector.findPos(img,setDraw=False)
    if len(lmList)!=0:
        #print(lmList)
        x1,y1=lmList[8][1:]   #tip of index finger
        x2,y2=lmList[12][1:]  #tip of middle finger
        
        
        
        #3.Check which fingers are up
        fingers=detector.fingersUp()
        #print(fingers)
        #a.Two fingers up->Selection mode
        if fingers[1] and fingers[2]:
            xp,yp=0,0
            print("Selection Mode")
            if(y1<142):       #checking if inside the header
                if(440<x1<590):
                    header=overlayList[0]
                    drawColor=(23,23,255)   #BGR format
                elif(680<x1<810):
                    header=overlayList[1]
                    drawColor=(171,74,0)
                elif(890<x1<1030):
                    header=overlayList[2]
                    drawColor=(3,244,241)
                elif(1110<x1<1230):
                    header=overlayList[3]
                    drawColor=(0,0,0)
            cv2.rectangle(img,(x1,y1-25),(x2,y2+25),drawColor,cv2.FILLED)
            
        
        
        #b.One finger up->Drawing mode
        if fingers[1] and fingers[2]==False:
            cv2.circle(img,(x1,y1),15,drawColor,cv2.FILLED)
            print("Drawing Mode")
            if(xp==0 and yp==0):
                xp,yp=x1,y1
            if drawColor==(0,0,0):
                cv2.line(img,(xp,yp),(x1,y1),drawColor,eraserThickness)
                cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,eraserThickness)
            else:
                cv2.line(img,(xp,yp),(x1,y1),drawColor,brushThickness)
                cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,brushThickness)
            xp,yp=x1,y1
    imgGray=cv2.cvtColor(imgCanvas,cv2.COLOR_BGR2GRAY) ##converting to gray image
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img,imgInv)
    img = cv2.bitwise_or(img,imgCanvas)
    
            
        
        
    
    #displaying the header image
    img[0:142,0:1280]=header
    
    cv2.imshow("Image",img)
    cv2.imshow("Canvas",imgCanvas)
    cv2.imshow("Inv",imgInv)
    cv2.waitKey(1)
    

['2.jpg', '3.jpg', '4.jpg', '5.jpg']
4
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Drawing Mode
